In [1]:
from qiskit import QuantumRegister
import numpy as np
from gymnasium.spaces import Box
from rl_qoc import QuantumEnvironment, BenchmarkConfig, GateTarget
from qiskit.circuit import QuantumCircuit, ParameterVector, Gate
from qiskit.circuit.library import CZGate, RXGate, XGate
from rl_qoc import (
    QEnvConfig,
    ExecutionConfig,
    ActionSpace
)
from rl_qoc.qibo import QiboConfig
from gymnasium.wrappers import ClipAction


def param_circuit(qc: QuantumCircuit, params: ParameterVector, qreg: QuantumRegister, **kwargs):
    target = kwargs["target"]
    gate: Gate = target.gate
    custom_gate = Gate(f"{gate.name}_cal", 2, params.params)
    qc.append(custom_gate, qreg)

    return qc


def gate_rule():
    pass


def get_backend():
    return "qibolab"


target = GateTarget(gate=XGate(), physical_qubits=[0])
action_space = ActionSpace(n_actions=2, low=[0.0, 16], high=[1.0, 160])
qibo_config = QiboConfig(
    "param_circuit",
    "get_backend",
    platform="dummy",
    physical_qubits=(0, 1),
    gate_rule="rx",
    parametrized_circuit_kwargs={"target": target},
    instruction_durations=None,
)
q_env_config = QEnvConfig(
    target=target,
    backend_config=qibo_config,
    action_space=action_space,
    benchmark_config=BenchmarkConfig(0),
    execution_config=ExecutionConfig(batch_size=32, sampling_paulis=50, n_shots=1000, n_reps=1),
)

env = QuantumEnvironment(q_env_config)

In [2]:
env.circuits[0].draw(output="mpl")

In [3]:
env.baseline_circuits[0].draw(output="mpl")

In [4]:
from rl_qoc import CustomPPO
from rl_qoc.agent import TrainFunctionSettings, TotalUpdates, TrainingConfig
from rl_qoc.helpers import load_from_yaml_file

file_name = "agent_config.yaml"

agent_config = load_from_yaml_file(file_name)

In [5]:
ppo = CustomPPO(agent_config, ClipAction(env))
total_updates = TotalUpdates(500)
# hardware_runtime = HardwareRuntime(300)
training_config = TrainingConfig(
    training_constraint=total_updates,
    target_fidelities=[0.999, 0.9999],
    lookback_window=10,
    anneal_learning_rate=True,
    std_actions_eps=1e-2,
)

train_function_settings = TrainFunctionSettings(
    plot_real_time=False,
    print_debug=False,
    num_prints=5,
    hpo_mode=False,
    clear_history=False,
)

In [ ]:
ppo.train(training_config, train_function_settings)